In [42]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import requests
import sncosmo

In [44]:
username = 'jnordin'
date = '2018-09-23'
maxz = 0.1
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 5          # A candidate need to have at least this many detections
maxage = 30         # If a detection has an age older than this, skip (stars,age). 
                    # Q: How can histories be older than 30days?
minfilters = 2      # Reported detections in at least this many filters
marshal_saveprogs = 'AMPEL'   # Candidates already in this program will not be scanned. Reg exp string, eg 'AMPEL|RCF'
marshal_savid = 42
logpath = '/home/jnordin/local/git/AmpelSlackCosmology/'
slackfilepath = '/home/jnordin/Downloads/'

# log
#logging.basicConfig(filename=filepath+'ztfcosmo_%s_%s.log'%(date,username))
logger = logging.getLogger(username)
handler = logging.FileHandler(logpath+'ztfcosmo_%s_%s.log'%(date,username))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# marshaltool


In [4]:
df_sum = pd.read_csv(slackfilepath+"Summary_%s.csv"%(date))
df_phot = pd.read_csv(slackfilepath+"Photometry_%s.csv"%(date))

In [5]:
logger.info("There were %s candidates reported %s"%(df_sum.shape[0],date))

INFO:jnordin:There were 183 candidates reported 2018-09-23


In [6]:
# Find already classified SNe
i_untyped = (df_sum["T2-classification"]== "None") | (df_sum["T2-classification"].isnull())
df_classified = df_sum[ ~i_untyped]
df_tmp = df_sum[ i_untyped]
logger.info("Out of these %s already have Marshal type and will be skipped."%len(df_classified))


INFO:jnordin:Out of these 47 already have Marshal type and will be skipped.


In [7]:
# Are there any remaining candidates with spectroscopic redshifts above limit
i_highz = ( (df_tmp['T2-NEDz_z']>maxz) | (df_tmp['T2-milliquas_redshift']>maxz)  | (df_tmp['T2-SDSS_spec_z']>maxz) ) 
df_classified = df_tmp[ i_highz]
df_tmp2 = df_tmp[~i_highz]
logger.info("%s have a catalog z larger than %s  and will be skipped."%(np.sum(i_highz),maxz))


INFO:jnordin:3 have a catalog z larger than 0.1  and will be skipped.


In [8]:
# Which are already saved in the marshal?
i_saved = df_tmp2["T2-programs"].str.contains(marshal_saveprogs, regex=True, na=False, flags=re.IGNORECASE)
df_saved = df_tmp2[i_saved]
df_candidates = df_tmp2[~i_saved]
logger.info("Detected %s already saved candidates and %s new"%(len(df_saved),len(df_candidates)))

INFO:jnordin:Detected 9 already saved candidates and 124 new


In [46]:
# Examine new candidates, build a list of the onces fulfilling basic criteria
sncandidates = []
for sne in df_candidates.iterrows():
    print( sne[1]["ztf_name"] )
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']
    if np.min(magpsf)> minpeakmag:
        msg = "%s: Skipping %s, never reached limit %s"%(date,sne[1]["ztf_name"],minpeakmag)
        logger.info(msg)
        print(msg)
        continue
    if len(magpsf)< mindet:
        msg = "%s: Skipping %s, less than %s detections"%(date,sne[1]["ztf_name"],mindet)
        logger.info(msg)
        print(msg)
        continue

    age = np.max(jd)-np.min(jd)
    if age>maxage:
        msg = "%s: Skipping %s, age %s longer than limit %s"%(date,sne[1]["ztf_name"],age,maxage)
        logger.info(msg)
        print(msg)
        continue
        
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    if len(np.unique(phot["fid"]))<minfilters:
        msg = "%s: Skipping %s, only %s filters"%(date,sne[1]["ztf_name"],len(np.unique(phot["fid"])))
        logger.info(msg)
        print(msg)
        continue
        
    sncandidates.append(sne[1]["ztf_name"])
    continue
        
    

print("Left with %s candidates to scan"%(len(sncandidates)))

INFO:jnordin:2018-09-23: Skipping ZTF18abwusgl, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18abslxpd, age 39.03981479955837 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abyenql, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18aaupjtd, age 46.06000000005588 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18ablwolq, only 1 filters
INFO:jnordin:2018-09-23: Skipping ZTF18aayaphx, age 46.996886500157416 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18aamhnti, age 49.95512729976326 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abnvotk, age 35.85531250014901 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abhpdpk, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abvvyro, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18abehrdw, age 45.064872700255364 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18absmsbm, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipp

ZTF18abwusgl
2018-09-23: Skipping ZTF18abwusgl, less than 5 detections
ZTF18abvnwzz
ZTF18abslxpd
2018-09-23: Skipping ZTF18abslxpd, age 39.03981479955837 longer than limit 30
ZTF18abyenql
2018-09-23: Skipping ZTF18abyenql, less than 5 detections
ZTF18aaupjtd
2018-09-23: Skipping ZTF18aaupjtd, age 46.06000000005588 longer than limit 30
ZTF18abecggb
ZTF18ablwolq
2018-09-23: Skipping ZTF18ablwolq, only 1 filters
ZTF18aayaphx
2018-09-23: Skipping ZTF18aayaphx, age 46.996886500157416 longer than limit 30
ZTF18aamhnti
2018-09-23: Skipping ZTF18aamhnti, age 49.95512729976326 longer than limit 30
ZTF18abnvotk
2018-09-23: Skipping ZTF18abnvotk, age 35.85531250014901 longer than limit 30
ZTF18abqboud
ZTF18abvhasp
ZTF18abhpdpk
2018-09-23: Skipping ZTF18abhpdpk, never reached limit 19.5
ZTF18abvvyro
2018-09-23: Skipping ZTF18abvvyro, less than 5 detections
ZTF18abehrdw
2018-09-23: Skipping ZTF18abehrdw, age 45.064872700255364 longer than limit 30
ZTF18absmsbm
2018-09-23: Skipping ZTF18absmsbm, nev

INFO:jnordin:2018-09-23: Skipping ZTF18abxishs, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18aazydub, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abmsmnc, age 40.04186340002343 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abxfcql, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18abytyjb, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abvwpnt, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18abtzisn, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abnvnqb, age 35.85486110020429 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abmsngt, age 40.0423494996503 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF17aabihdn, age 41.002847200259566 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18aampiwv, age 46.89600689988583 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18aboncwu, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18a

ZTF18abxishs
2018-09-23: Skipping ZTF18abxishs, less than 5 detections
ZTF18abwcgyo
ZTF18abslxhz
ZTF18aazydub
2018-09-23: Skipping ZTF18aazydub, never reached limit 19.5
ZTF18abmsmnc
2018-09-23: Skipping ZTF18abmsmnc, age 40.04186340002343 longer than limit 30
ZTF18abxfcql
2018-09-23: Skipping ZTF18abxfcql, less than 5 detections
ZTF18abytyjb
2018-09-23: Skipping ZTF18abytyjb, never reached limit 19.5
ZTF18abvwpnt
2018-09-23: Skipping ZTF18abvwpnt, less than 5 detections
ZTF18abtzisn
2018-09-23: Skipping ZTF18abtzisn, never reached limit 19.5
ZTF18abnvnqb
2018-09-23: Skipping ZTF18abnvnqb, age 35.85486110020429 longer than limit 30
ZTF18abmsngt
2018-09-23: Skipping ZTF18abmsngt, age 40.0423494996503 longer than limit 30
ZTF17aabihdn
2018-09-23: Skipping ZTF17aabihdn, age 41.002847200259566 longer than limit 30
ZTF18aampiwv
2018-09-23: Skipping ZTF18aampiwv, age 46.89600689988583 longer than limit 30
ZTF18abrwrch
ZTF18abwdcdv
ZTF18aboncwu
2018-09-23: Skipping ZTF18aboncwu, never reached

INFO:jnordin:2018-09-23: Skipping ZTF18abmsmtt, age 40.04281250014901 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abucoyd, only 1 filters
INFO:jnordin:2018-09-23: Skipping ZTF18aaakpmq, only 1 filters
INFO:jnordin:2018-09-23: Skipping ZTF18aaqzomm, age 36.91467590024695 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abwlsoi, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18abyckee, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18abxazsz, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abyckme, less than 5 detections
INFO:jnordin:2018-09-23: Skipping ZTF18absnmdd, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abytfxc, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abxqzih, only 1 filters
INFO:jnordin:2018-09-23: Skipping ZTF18abfkzjd, age 45.82319450005889 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18aaxudip, age 48.019444399513304 longer than limit 30
INFO:jnordin:2


ZTF18abmsmtt
2018-09-23: Skipping ZTF18abmsmtt, age 40.04281250014901 longer than limit 30
ZTF18abucoyd
2018-09-23: Skipping ZTF18abucoyd, only 1 filters
ZTF18abuxpun
ZTF18aaowwgf
ZTF18aaakpmq
2018-09-23: Skipping ZTF18aaakpmq, only 1 filters
ZTF18aaqzomm
2018-09-23: Skipping ZTF18aaqzomm, age 36.91467590024695 longer than limit 30
ZTF18abwlsoi
2018-09-23: Skipping ZTF18abwlsoi, less than 5 detections
ZTF18abyckee
2018-09-23: Skipping ZTF18abyckee, less than 5 detections
ZTF18abwfmot
ZTF18abxazsz
2018-09-23: Skipping ZTF18abxazsz, never reached limit 19.5
ZTF18abyckme
2018-09-23: Skipping ZTF18abyckme, less than 5 detections
ZTF18absnmdd
2018-09-23: Skipping ZTF18absnmdd, never reached limit 19.5
ZTF18abytfxc
2018-09-23: Skipping ZTF18abytfxc, never reached limit 19.5
ZTF18abxqzih
2018-09-23: Skipping ZTF18abxqzih, only 1 filters
ZTF18abfkzjd
2018-09-23: Skipping ZTF18abfkzjd, age 45.82319450005889 longer than limit 30
ZTF18aaxudip
2018-09-23: Skipping ZTF18aaxudip, age 48.01944439951

INFO:jnordin:2018-09-23: Skipping ZTF18abvxbfh, only 1 filters
INFO:jnordin:2018-09-23: Skipping ZTF18abitwua, age 49.9795834007673 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abwtops, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abpfbri, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abwklar, only 1 filters
INFO:jnordin:2018-09-23: Skipping ZTF18abytwah, never reached limit 19.5
INFO:jnordin:2018-09-23: Skipping ZTF18abcleke, age 35.8923842003569 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abgqvwv, age 50.95193289965391 longer than limit 30
INFO:jnordin:2018-09-23: Skipping ZTF18abuakku, never reached limit 19.5


2018-09-23: Skipping ZTF18abvxbfh, only 1 filters
ZTF18abitwua
2018-09-23: Skipping ZTF18abitwua, age 49.9795834007673 longer than limit 30
ZTF18abukmty
ZTF18abbmenz
ZTF18abwtops
2018-09-23: Skipping ZTF18abwtops, never reached limit 19.5
ZTF18abpfbri
2018-09-23: Skipping ZTF18abpfbri, never reached limit 19.5
ZTF18abwklar
2018-09-23: Skipping ZTF18abwklar, only 1 filters
ZTF18abytwah
2018-09-23: Skipping ZTF18abytwah, never reached limit 19.5
ZTF18abcleke
2018-09-23: Skipping ZTF18abcleke, age 35.8923842003569 longer than limit 30
ZTF18abgqvwv
2018-09-23: Skipping ZTF18abgqvwv, age 50.95193289965391 longer than limit 30
ZTF18abubmjo
ZTF18abuakku
2018-09-23: Skipping ZTF18abuakku, never reached limit 19.5
ZTF18abwdhgw
Left with 41 candidates to scan


In [10]:
# Function to make stupid plot of a SN lightcurve
def plotty(i):
    """
    Quick plot of SN nbr k
    i : {0:undecided, 1:submit, 2:wait 3:bogus}
    """
    
    global snnbr
    
    
    # Evaluate reply
    snname = sncandidates[snnbr]
    if i == 'Wait':
        logger.info("Waiting for further obs for SN %s %s"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'Submit':
        logger.info("Sending SN %s %s for possible follow-up"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'Garbage':
        logger.info("I never want to see SN %s %s again!"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'GoBack':
        logger.info("I want to scan some more, go back!")
        snnbr -= 1
    elif i=='Nothing':
        # Lets do nothing
        pass
    
    # Are we done?
    if snnbr==len(sncandidates):
        print( "Seems like we are all done. How do we exit?")
        return False
    
    
    # Reset    
    snname = sncandidates[snnbr]
    

    phot = df_phot[ df_phot["ztf_name"]==snname ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']


    plt.title(snname + ' ' + str(snnbr) + ' out of '+str(len(sncandidates)))
    plt.plot(jd[filters==1]-Time.now().jd,magpsf[filters==1],'go')
    plt.plot(jd[filters==2]-Time.now().jd,magpsf[filters==2],'ro')
    plt.plot(jd[filters==3]-Time.now().jd,magpsf[filters==3],'ko')
    plt.gca().invert_yaxis()
    plt.show()
    wiggy.value = 'Nothing'
    
    return (i)
wiggy = widgets.RadioButtons(
    options=['Nothing','Wait', 'Submit', 'Garbage','GoBack'],
    value='Nothing',
    description='Action:',
    disabled=False
)
y = interactive(plotty,i = wiggy)

In [ ]:
# Presumably you want to scan from the first SN, but you can change this and run display again to revisit something
# You cant jump ahead in the list though
snnbr = 0


In [ ]:
# This is the all important list of decisions you have made
decisions = {}

In [ ]:
# This is the scanning box! 
# You have three choices (Nothing is not a choice and GoBack steps lets you go back in order)
# - Wait : Select this if the transient is rizing and might get into RCF range (~<18.7)
# - Submit : Should probably get a spectrum (you do not need to worry about where just yet)
# - Garbage : Variable star or clearly non SNIa. Evanetually these will be rejected and never more show up
display(y)

In [39]:
# Lets try to ingest the sne that were previously candidates.
def ingest(avroid, ProgramList,programidx):
    """
    Ingesting some avro into a program described by a marshaltools ProgramList object
    """
    
    marshal_root = 'http://skipper.caltech.edu:8080/cgi-bin/growth/'
    ingest_url = marshal_root + 'ingest_avro_id.cgi'
    
    postdata = {'programidx':str(programidx),'avroid':str(avroid)}
    request = requests.post(ingest_url,data=postdata, auth=(ProgramList.user,ProgramList.passwd))
    return request

In [40]:
ingestoldcand_start = Time.now()
print("Starting ingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_start))
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue
    #if sncandidates[snnbr] in pl.candidates.keys(): continue
    # Whic h
    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    print("Trying to ingest %s through avroid %s"%(sncandidates[snnbr],avroid))
    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        print('... already a candidate with id %s'%(canddate['candid']))
        continue
    
    print("now we need to be anal")
    request = ingest(avroid,pl,4)
    print(request.text)
    
    
#    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    #print(canddate["candid"])
    #print(canddate)
ingestoldcand_end = Time.now()
print("Stoppedingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_end))

DEBUG:marshaltools.ProgramList:found source ZTF18abucvbf among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abwlpzf among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18absnsqx among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18absnsqx among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18absloer among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18absloer among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abslxhz among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18abrwrch among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find it among candidates either.


Starting ingesting at 2018-09-25 14:00:55.834780 (useful if you want to check scanning page directly)
Trying to ingest ZTF18abucvbf through avroid 628193691815015016
... already a candidate with id 616223801815015049
Trying to ingest ZTF18abwlpzf through avroid 629171804015015017
... already a candidate with id 629171804015015017
Trying to ingest ZTF18absnsqx through avroid 629307755715015003
... already a candidate with id 632254845715015004
Trying to ingest ZTF18absloer through avroid 628448994015015015
... already a candidate with id 631360694015015019
Trying to ingest ZTF18abslxhz through avroid 628189953215015068
... already a candidate with id 621289093215015043
Trying to ingest ZTF18abrwrch through avroid 627195524515015019
now we need to be anal


DEBUG:marshaltools.ProgramList:found source ZTF18abwdcdv among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18abwktmm among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abwktmm among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abvnqqk among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abtogdl among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18abvcrsv among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abvcrsv among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18abtghbi among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abtghbi among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abukmty among saved sources of progr


<br> programid: 42 <br> avroid: 627195524515015019 <br>
Status: 422  Invalid Input



No avro packet found with specified id.

Trying to ingest ZTF18abwdcdv through avroid 628179242615015015
... already a candidate with id 622199782615015054
Trying to ingest ZTF18abwktmm through avroid 630400414415015002
... already a candidate with id 632499504415015005
Trying to ingest ZTF18abvnqqk through avroid 628190453615015006
... already a candidate with id 622215183615015021
Trying to ingest ZTF18abtogdl through avroid 628182502715015023
... already a candidate with id 615259722715015081
Trying to ingest ZTF18abvcrsv through avroid 630201452915015040
... already a candidate with id 632276142915015084
Trying to ingest ZTF18abtghbi through avroid 630506630415015015
... already a candidate with id 632411010415015002
Trying to ingest ZTF18abukmty through avroid 628302610915015005
... already a candidate with id 618275210915015001
Trying to ingest ZTF18abwdhgw through avroid 630268591115015039
...

In [41]:
# Now fun starts for real. Lets download all the candidates and check whether we find all the candidate ids there
# (cause are the only that are useful for saving). This will take a while.
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)

DEBUG:marshaltools.ProgramList:listing accessible programs
DEBUG:marshaltools.ProgramList:Starting list_programs.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_programs.cgi?None
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
DEBUG:marshaltools.ProgramList:Starting list_program_sources.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_program_sources.cgi?programidx=4&getredshift=1&getclassification=1
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Loaded 172 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Getting scanning page transient for program AMPEL Test between 2018-03-01 00:00:00 and 2018-09-25 14:01:17 using dt: 120.00 h
INFO:marshaltools.ProgramList:Downloading candidates. Iteration 

DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-05-30 00:00:00 and 2018-06-04 00:00:00
DEBUG:marshaltools.ProgramList:Query from 2018-04-15 00:00:00.000 to 2018-04-20 00:00:00.000 returned 0 candidates. Total: 0
DEBUG:marshaltools.ProgramList:Starting list_candidates.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-03-16+00%3A00%3A00&enddate=2018-03-21+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-06-04 00:00:00 and 2018-06-09 00:00:00
DEBUG:marshaltools.ProgramList

DEBUG:marshaltools.ProgramList:Query from 2018-06-14 00:00:00.000 to 2018-06-19 00:00:00.000 returned 1 candidates. Total: 1
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-07-29 00:00:00 and 2018-08-03 00:00:00
DEBUG:marshaltools.ProgramList:Starting list_candidates.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:Query from 2018-06-09 00:00:00.000 to 2018-06-14 00:00:00.000 returned 0 candidates. Total: 1
DEBUG:marshaltools.ProgramList:Starting list_candidates.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-05-20+00%3A00%3A00&enddate=2018-05-25+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingest

DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 77 sources
DEBUG:marshaltools.ProgramList:Query from 2018-07-04 00:00:00.000 to 2018-07-09 00:00:00.000 returned 77 candidates. Total: 302
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-08-23+00%3A00%3A00&enddate=2018-08-28+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 186 sources
DEBUG:marshaltools.ProgramList:Query from 2018-08-23 00:00:00.000 to 2018-08-28 00:00:00.000 returned 186 candidates. Total: 488
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-08-03+00%3A00%3A00&enddate=2018-08-08+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrie

In [ ]:
# We first need to make sure these are ingested so we can get candidate IDs
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue
    
    snname = sncandidates[snnbr]
    
    if not snname in pl.candidates.keys():
        print( "%s not among candidates though submitted"%(snname) )
        continue
    else:
        if not 'candid' in pl.candidates[sncandidates[snnbr]].keys():
            print( "All candidates should have candid, right?" )
            sys.exit('yee')
            
        print("%s %s"%(sncandidates[snnbr], pl.candidates[sncandidates[snnbr]]['candid']))
    
    #print(candidates[sncandidates[snnbr]].keys())
    
    #weblink= path % (sncandidates[snnbr], marshal_savid)
    #print(weblink)
    
    print(pl.candidates[sncandidates[snnbr]].keys())
    continue


In [12]:
from marshaltools.gci_utils import growthcgi

In [ ]:
def get_marshal_html(weblink, max_attempts=5, marshalusr=None,marshalpwd=None):

    auth = requests.auth.HTTPBasicAuth(marshalusr, marshalpwd)
    attempts = 1

	while attempts<max_attempts:	
		try:
			reponse = requests.get(weblink, auth=auth, timeout=30+(60*attempts-1))
			return reponse.text
		except Exception as e:
			log.error('Sergeant.get_marshal_html(): problem with url: {0} \n{1} \nthis attempt number {2}, {3} left'.format(weblink, e, attempts, max_attempts-attempts))
			time.sleep(3)
		attempts+=1

	log.error('giving up')
	raise(requests.exceptions.ConnectionError) 

In [ ]:
weblink = 'http://skipper.caltech.edu:8080/cgi-bin/growth/save_cand_growth.cgi?candid=ZTF18abecggb&program=4'

In [ ]:
foo = get_marshal_html(weblink,marshalusr=os.environ['GROWTHUSER'], marshalpwd=os.environ['GROWTHPWD'])

In [ ]:
print(foo)

In [11]:
decisions = {0: 'Garbage', 1: 'Garbage', 2: 'Garbage', 3: 'Garbage', 4: 'Garbage', 5: 'Wait', 6: 'Wait', 7: 'Garbage', 8: 'Submit', 9: 'Submit', 10: 'Garbage', 11: 'Garbage', 12: 'Garbage', 13: 'Garbage', 14: 'Garbage', 15: 'Submit', 16: 'Submit', 17: 'Garbage', 18: 'Submit', 19: 'Submit', 20: 'Submit', 21: 'Garbage', 22: 'Garbage', 23: 'Garbage', 24: 'Garbage', 25: 'Garbage', 26: 'Garbage', 27: 'Garbage', 28: 'Garbage', 29: 'Garbage', 30: 'Garbage', 31: 'Garbage', 32: 'Garbage', 33: 'Garbage', 34: 'Garbage', 35: 'Garbage', 36: 'Garbage', 37: 'Garbage', 38: 'Submit', 39: 'Garbage', 40: 'Submit', 41: 'Submit', 42: 'Submit', 43: 'Submit', 44: 'Garbage', 45: 'Garbage', 46: 'Submit', 47: 'Garbage', 48: 'Garbage', 49: 'Garbage', 50: 'Submit'}

In [31]:
request.text

'\nStatus: 422  Invalid Input\n\n\n\nAvro packet id not supplied.\n'